# Lyman alpha fitting

The equation for the cross section is (Diplas \& Savage 1994)

$ \sigma(\lambda) = 4.26e^{-20} \text{cm}^2 / [6.04e^{-10}+ (\lambda - \lambda_0)^2] $

where $\lambda_0 = 1215.67 \AA$. We might need a velocity doppler shift parameter for $\lambda_0$. The model for the spectrum, with $N = N(\text{H I})$ as a parameter, is written as

$ f_m(\lambda; N) = f_c(\lambda) \exp(-N \sigma(\lambda)) $.


The function $f_c(\lambda)$ describes the continuum, the flux without any lines. It is usually a constant or 1st order approximation, determined by fitting or sometimes by eye. $N(\text{HI})$ can be found by minimizing a $\chi^2$ function

$ \chi^2(N) = \frac{1}{n - 1} \sum_i \sigma
_c^{-1} (f_i - f_m(\lambda_i; N))^2$. 

Here, a suitable noise model $\sigma_c$ needs to be chosen. In DS94, $\sigma_c = \text{constant} \times \exp(\sigma(\lambda_i) N)$. This is not an error on the measurement, but rather a factor to treat the other features of the spectrum as noise. For the constant, they use the RMS deviation with respect to $f_c$, over a wavelength range outside the line, deemed to have similar noise as the line wings. 

The workflow goes as follows:
1. Find a suitable approximation for the continuum.
2. Find a suitable noise level with respect to the continuum.
2. Mask wavelengths that have troublesome features.
3. Optimize the $\chi^2$ function for the remaining data points $(\lambda_i, f_i)$, minimizing the differences between $f_i$ and $f_m(\lambda_i)$.


In [ ]:
def sigma(l):
    l0 = 1215.67
    return 4.26e-20 / (6.04e-10 + np.square(l - l0))
    
def fmodel(fcontinuum, NHI):
    return lambda l : fcontinuum(l) * np.exp(-NHI * sigma(l))

def chi2(NHI, fc, sigma_c, wavs, fs):
    extinctions = np.exp(NHI * sigma(l))
    deltas = fc(wavs) - fs * extinctions
    sigmas = sigma_c * extinctions
    chi2 = np.square((deltas / sigmas)).sum() / (len(deltas) - 1)
    return chi2


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 226

def prepare_axes():
    plt.xlabel('wavelength')
    plt.ylabel('flux (erg cm$^{-2}$s$^{-1}\\AA^{-1}$)')

def plot_iue(table):
    plt.plot(table['WAVELENGTH'], table[data_column_name])
    prepare_axes()
    
def plot_iue_order(table, index):
    data_column_name = 'ABS_CAL'
    flux = table[index][data_column_name]
    wavs = [table[index]['WAVELENGTH'] + table[index]['DELTAW'] * i for i in range(len(flux))]
#    dq = table[index]['QUALITY']
 #   good = dq == -2  
  #  plt.plot(wavs[good], flux[good])
    plt.plot(wavs, flux)
    
def plot_stis_order(table, index):
    wavs = table[index]['WAVELENGTH']
    data_column_name = 'FLUX'
    flux = table[index][data_column_name]
    plt.plot(wavs, flux)
    
def plot_function_at_lya(f, ylabel='f(x)', *args):
    x = np.linspace(1100, 1300, 1000)
    y = f(x, *args)
    plt.xlabel('wavelength')
    

## Data exploration

In [ ]:
# open some data
from astropy.io import fits
from astropy.table import Table
from pathlib import Path

In [ ]:
def sort_by_wav(wavs, flux):
    idxs = np.argsort(wavs)
    return wavs[idxs], flux[idxs]

### STIS data looks like this

In [ ]:
spectrum_file = '/Users/dvandeputte/Projects/FUSE H2/plotting_git/data/HD094493/mastDownload/HST/o54306010/o54306010_x1d.fits'
fits.info(spectrum_file)

In [ ]:
t = Table.read(spectrum_file)
t

In [ ]:
import numpy as np
def stis_merge_orders(t):
    """
    Put the data from all spectral orders of a set of STIS data into one big array
    
    Returns
    -------
        allwavs, allflux: ndarray, wavelengths and fluxes, sorted by wavelength
    """
    allwavs = np.concatenate(t['WAVELENGTH'])
    allflux = np.concatenate(t['FLUX'])
    return sort_by_wav(allwavs, allflux)

In [ ]:
# try plotting this
x, y = stis_merge_orders(t)
plt.plot(x, y)

### IUE high res like this

In [ ]:
spectrum_file = '/Users/dvandeputte/Projects/FUSE H2/plotting_git/data/HD094493/swp49770.mxhi.gz'
fits.info(spectrum_file)

In [ ]:
t = Table.read(spectrum_file)
t

In [ ]:
# check the contents of one row
row = 1
print(t[row]['WAVELENGTH'])
t[row]['NET'].shape
dq = t[row]['QUALITY']
dq == -2

In [ ]:
def iue_h_merge_orders(t):
    
    def iue_wavs(i):
        return t[i]['WAVELENGTH'] + t[i]['DELTAW'] * np.arange(t[i]['NPOINTS'])
    
    def iue_flux(i):
        pixrange = slice(t[i]['STARTPIX'], t[i]['STARTPIX'] + t[i]['NPOINTS'])
        return t[i]['ABS_CAL'][pixrange]
    
    allwavs = np.concatenate([iue_wavs(i) for i in range(len(t))])
    allflux = np.concatenate([iue_flux(i) for i in range(len(t))])
    
    return sort_by_wav(allwavs, allflux)

In [ ]:
allwavs, allflux = iue_h_merge_orders(t)
plt.plot(allwavs, allflux)